In [1]:
import torch

In [2]:
print("This is will complete in some time.....")

This is will complete in some time.....


In [3]:
import optuna

# First we'll take a look at what is optuna and how to use this
# Bayesian search
# accuray = fun(max_depth , n_estimate) , try to find this fucntion
# as we get more and more data, we'll estimte this fucntion
# grid search and random seach are dumb

In [4]:
# -------- IMP Term ----------
# study - whole process that has mutiple trails
# trial - trying diff combination , one single study
# Trail paramter - hyperparamter during a single trial
# objective fucntion - function that we are typing to estimate
# sample - bassed on the prevous fucntion, what we should try next

In [5]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder , OneHotEncoder , StandardScaler
from sklearn.impute import SimpleImputer 

In [6]:
df = pd.read_csv("Dataset/Titanic/train.csv")
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.50,NaN,S
853,854,1,1,"Lines, Miss. Mary Conover",female,16.0,0,1,PC 17592,39.40,D28,S
141,142,1,3,"Nysten, Miss. Anna Sofia",female,22.0,0,0,347081,7.75,NaN,S
770,771,0,3,"Lievens, Mr. Rene Aime",male,24.0,0,0,345781,9.50,NaN,S
883,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.50,NaN,S


In [7]:
# Preprocss the datset
df = df.drop(columns= ['Name' , 'Cabin' ,"PassengerId" , "Ticket"])

# Split the dataset
y = df['Survived']
X = df.drop(columns= ['Survived'])
X_train , x_test , y_train , y_test = train_test_split(X, y , test_size= 0.2 , random_state= 42)
X_train.shape , x_test.shape

# impuation for Age
Si = SimpleImputer(strategy= "mean")
X_train['Age'] = Si.fit_transform(X_train[['Age']])
x_test['Age'] = Si.transform(x_test[["Age"]])

# Impuation for Embarked
X_train = X_train.fillna({'Embarked': 'missing'})  # Replace with 'missing' instead of dropping
x_test = x_test.fillna({'Embarked': 'missing'})

# OHE
object_columns = X_train.select_dtypes(include=['object']).columns.tolist()
ohe = OneHotEncoder(sparse_output= False)

X_train_ohe_encoder = ohe.fit_transform(X_train[object_columns])
x_test_ohe_encoder = ohe.transform(x_test[object_columns])

X_train_ohe_df = pd.DataFrame(X_train_ohe_encoder , columns= ohe.get_feature_names_out(object_columns), index= X_train.index)
x_test_ohe_df = pd.DataFrame(x_test_ohe_encoder , columns= ohe.get_feature_names_out(object_columns) , index= x_test.index)

X_train = pd.concat([X_train , X_train_ohe_df] , axis= 1)
x_test = pd.concat([x_test , x_test_ohe_df] , axis= 1)

X_train = X_train.drop(object_columns , axis= 1)
x_test = x_test.drop(object_columns , axis= 1)

# Scale Values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
x_test = scaler.transform(x_test)


In [8]:
print(X_train.shape , y_train.shape)

(712, 11) (712,)


In [9]:
# defining out workflow 
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier 

def objetive(trial):
    
    n_estimators = trial.suggest_int("n_estimators" , 50 , 100)
    max_depth = trial.suggest_int("max_depth" , 3 , 20)
    
    model = RandomForestClassifier(n_estimators= n_estimators, max_depth= max_depth , random_state= 42)
    
    score = cross_val_score(model , X_train , y_train , cv= 3 , scoring= "accuracy").mean()
    
    return score


In [10]:
study = optuna.create_study(direction= "maximize" , sampler= optuna.samplers.TPESampler())
study.optimize(objetive , n_trials= 50)

[I 2025-01-14 19:07:38,849] A new study created in memory with name: no-name-30be1674-c00d-4946-a85c-74053322c589
[I 2025-01-14 19:07:39,413] Trial 0 finished with value: 0.8216324504485338 and parameters: {'n_estimators': 76, 'max_depth': 7}. Best is trial 0 with value: 0.8216324504485338.
[I 2025-01-14 19:07:40,074] Trial 1 finished with value: 0.8047902705385952 and parameters: {'n_estimators': 98, 'max_depth': 12}. Best is trial 0 with value: 0.8216324504485338.
[I 2025-01-14 19:07:40,433] Trial 2 finished with value: 0.7977638312709049 and parameters: {'n_estimators': 68, 'max_depth': 17}. Best is trial 0 with value: 0.8216324504485338.
[I 2025-01-14 19:07:40,984] Trial 3 finished with value: 0.7963455424363839 and parameters: {'n_estimators': 98, 'max_depth': 15}. Best is trial 0 with value: 0.8216324504485338.
[I 2025-01-14 19:07:41,287] Trial 4 finished with value: 0.8230448297462445 and parameters: {'n_estimators': 61, 'max_depth': 5}. Best is trial 4 with value: 0.82304482974

In [11]:
print(f"Best accuarcy score: {study.best_trial.value}")
print(f"Best parameter score: {study.best_trial.params}")

Best accuarcy score: 0.8314777387748348
Best parameter score: {'n_estimators': 83, 'max_depth': 4}


In [12]:
# Train the model now
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(**study.best_trial.params)

model.fit(X_train , y_train)
y_pred = model.predict(x_test)

print(f"accuacy on test daat is : {accuracy_score(y_pred , y_test)}")

accuacy on test daat is : 0.8100558659217877


In [13]:
# Doing grid search

search_space = {
    "n_estimators" : [20 , 40 , 20 , 80],
    "max_depth" : [5 , 10 , 15 , 20]
}

In [14]:
study2 = optuna.create_study(direction= "maximize" , sampler= optuna.samplers.GridSampler(search_space))
study2.optimize(objetive)

[I 2025-01-14 19:07:53,938] A new study created in memory with name: no-name-bcd30df2-814f-4dd0-ad03-407387cff7c1
c:\Users\singh\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\samplers\_grid.py:199: UserWarning: The value `40` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntDistribution(high=100, log=False, low=50, step=1)`.
  warnings.warn(
[I 2025-01-14 19:07:54,131] Trial 0 finished with value: 0.8202377997612548 and parameters: {'n_estimators': 40, 'max_depth': 5}. Best is trial 0 with value: 0.8202377997612548.
c:\Users\singh\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\samplers\_grid.py:199: UserWarning: The value `20` is out of range of the parameter `n_estimators`. The value will be used but the actual distribution is: `IntDistribution(high=100, log=False, low=50, step=1)`.
  warnings.warn(
[I 2025-01-14 19:07:54,212] Trial 1 finished with value: 0.8117989811958539 and parameters: 

In [15]:
# model with grid search
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(**study2.best_trial.params)

model.fit(X_train , y_train)
y_pred = model.predict(x_test)

print(f"accuacy on test data is : {accuracy_score(y_pred , y_test)}")

accuacy on test data is : 0.8100558659217877


In [17]:
from optuna.visualization import plot_intermediate_values , plot_optimization_history , plot_parallel_coordinate , plot_slice , plot_contour , plot_param_importances

In [18]:
plot_optimization_history(study)

In [19]:
plot_parallel_coordinate(study).show()

In [20]:
plot_slice(study).show()

In [21]:
plot_contour(study).show()

In [22]:
plot_param_importances(study).show()

In [23]:
# Define by run --> for dynamic search space
# 1 hyperparamter --> best algo and best hyperparamter 


In [26]:
# HYyperparametr on algorithms
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

def objective_algo(trial: optuna.trial):
    
    classifier_name = trial.suggest_categorical("classifier" , ["SVM" , "RandomForest" , "GradientBoosting"])
    
    if classifier_name == "SVM":
        # SVM hyperparameter
        
        C = trial.suggest_float("C" , 0.1 , 100 )
        kernal = trial.suggest_categorical("kernal" , ["linear" , "rbf" , "poly" , "sigmoid"])
        gamma = trial.suggest_categorical("gamma" , ["scale" , "auto"])
        
        model = SVC(C= C , kernel= kernal , gamma= gamma , random_state= 42)
        
    elif classifier_name == "RandomForest": 
        n_estimators = trial.suggest_int("n_estimators" , 50 , 100)
        max_depth = trial.suggest_int("max_depth" , 3 , 20)

        model = RandomForestClassifier(n_estimators= n_estimators, max_depth= max_depth , random_state= 42)
        
    elif classifier_name == "GradientBoosting":
        
        n_estimators = trial.suggest_int("n_estimators" , 50 , 300)
        learning_rate = trial.suggest_float("learning_rate" , 0.01 , 0.3)
        max_depth = trial.suggest_int("max_depth" , 3, 20)
        
        model = GradientBoostingClassifier(n_estimators= n_estimators , learning_rate= learning_rate , max_depth= max_depth , random_state=42)
        
    
    # perform cross validation and return mean accuracy
    score = cross_val_score(model , X_train , y_train , cv= 3 , scoring='accuracy').mean()
    return score   
        

In [27]:
study3 = optuna.create_study(direction="maximize")
study3.optimize(objective_algo , n_trials= 100)

[I 2025-01-14 19:49:52,949] A new study created in memory with name: no-name-e2b156ee-e043-43f1-8ec7-b3776cba14d4
[I 2025-01-14 19:49:53,238] Trial 0 finished with value: 0.7907492110768358 and parameters: {'classifier': 'SVM', 'C': 85.23028410911094, 'kernal': 'linear', 'gamma': 'auto'}. Best is trial 0 with value: 0.7907492110768358.
[I 2025-01-14 19:49:53,259] Trial 1 finished with value: 0.6868477348745405 and parameters: {'classifier': 'SVM', 'C': 84.31340315424312, 'kernal': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.7907492110768358.
[I 2025-01-14 19:49:53,519] Trial 2 finished with value: 0.8160183904785544 and parameters: {'classifier': 'RandomForest', 'n_estimators': 95, 'max_depth': 8}. Best is trial 2 with value: 0.8160183904785544.
[I 2025-01-14 19:49:55,461] Trial 3 finished with value: 0.7739011216300867 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 156, 'learning_rate': 0.2830358530350242, 'max_depth': 11}. Best is trial 2 with value

In [28]:
study3.best_params

{'classifier': 'RandomForest', 'n_estimators': 69, 'max_depth': 4}

In [32]:
study3.trials_dataframe()["params_classifier"].value_counts()

params_classifier
RandomForest        79
SVM                 11
GradientBoosting    10
Name: count, dtype: int64

In [34]:
study3.trials_dataframe().groupby('params_classifier')["value"].mean()

params_classifier
GradientBoosting    0.780767
RandomForest        0.818725
SVM                 0.774143
Name: value, dtype: float64